# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

One of the most basic objects in this approach are unevaluated *symbols*:

In [1]:
:a  # "the symbol a"

:a

In [2]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [3]:
eval(:a)

LoadError: UndefVarError: a not defined

`a` must be defined for this to work:

In [4]:
a = 3

3

In [5]:
eval(:a)

3

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Everything is a symbol:

In [6]:
:+, :sin

(:+,:sin)

In [7]:
typeof(:+)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [8]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [9]:
typeof(ex)

Expr

In [10]:
b = 7
eval(ex)

10

An expression is just a Julia object, so we can introspect (find out information about it):

In [11]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b
  typ: Any


In [13]:
dump(:(f(a,b)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol f
    2: Symbol a
    3: Symbol b
  typ: Any


In [12]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3
  typ: Any


In [ ]:
# ex.<TAB>

In [14]:
ex.head

:call

In [15]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

In [ ]:
ex.typ

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [16]:
parse("a + b")

:(a + b)

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [17]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [18]:
ex.args

2-element Array{Any,1}:
 :sin             
 :(3a + 2 * b ^ 2)

In [19]:
ex.args[2].args

3-element Array{Any,1}:
 :+          
 :(3a)       
 :(2 * b ^ 2)

In [21]:
ex.args[2].args[2].args

3-element Array{Any,1}:
  :*
 3  
  :a

In [22]:
ex.args[2].args[3].args

3-element Array{Any,1}:
  :*      
 2        
  :(b ^ 2)

In [23]:
ex.args[2].args[3].args[3].args

3-element Array{Any,1}:
  :^
  :b
 2  

In [24]:
ex.args[1] = :cos

:cos

In [25]:
ex

:(cos(3a + 2 * b ^ 2))

In [26]:
blk = quote
    println("Hello")
end

quote  # In[26], line 2:
    println("Hello")
end

In [27]:
eval(blk)

Hello


In [28]:
dump(blk)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 2
        2: Symbol In[26]
      typ: Any
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "Hello"
      typ: Any
  typ: Any


In [29]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :( # In[26], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [31]:
blk

quote  # In[26], line 2:
    println("Hello")
    println("AFTER")
end

In [30]:
eval(blk)

Hello
AFTER


In [32]:
unshift!(blk.args, :(println("BEFORE")))

4-element Array{Any,1}:
 :(println("BEFORE")) 
 :( # In[26], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [33]:
eval(blk)

BEFORE
Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [ ]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

In [ ]:
eval(ex2)
y

In [ ]:
eval(ex2)
z

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [36]:
Meta.show_sexpr(ex)

(:call, :cos, (:call, :+, (:call, :*, 3, :a), (:call, :*, 2, (:call, :^, :b, 2))))

Another way of seeing the structure is with `dump`:

In [ ]:
dump(ex2)

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately.

In [39]:
str = "hello"
eval(quote
    println($(2+3))
end)

5


In [40]:
fnames = [ Symbol("func$i") for i=1:2]

2-element Array{Symbol,1}:
 :func1
 :func2

In [41]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = $i
end

In [42]:
func1(0)

1

In [43]:
func2(0)

2

# Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [44]:
@time sin(10)

  0.005726 seconds (1.73 k allocations: 83.397 KB)


-0.5440211108893698

A trivial example of defining a macro is the following, which runs whatever code it is passed two times. The `$` sign is used to interpolate the value of the expression (similar to its usage for string interpolation):

In [45]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [46]:
x = 0
@twice println(x += 1)

1
2


In [ ]:
ex = :(@twice println(x += 1))

In [ ]:
eval(ex)

In [ ]:
typeof(ex)

We can see what effect the macro actually has using `macroexpand`:

In [47]:
macroexpand(:(@twice println(x += 1)))

quote  # In[45], line 3:
    println(x += 1) # In[45], line 4:
    println(x += 1)
end

In [48]:
macroexpand(:(@time sin(10)))

quote  # util.jl, line 182:
    local #6#stats = (Base.gc_num)() # util.jl, line 183:
    local #8#elapsedtime = (Base.time_ns)() # util.jl, line 184:
    local #7#val = sin(10) # util.jl, line 185:
    #8#elapsedtime = (Base.time_ns)() - #8#elapsedtime # util.jl, line 186:
    local #9#diff = (Base.GC_Diff)((Base.gc_num)(),#6#stats) # util.jl, line 187:
    (Base.time_print)(#8#elapsedtime,#9#diff.allocd,#9#diff.total_time,(Base.gc_alloc_count)(#9#diff)) # util.jl, line 189:
    #7#val
end

In [49]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [53]:
time() = 0
@mytime (sleep(1); "done")

1.0061230659484863 seconds elapsed


"done"

----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

In [ ]:
@until x == 0 begin
end

In [54]:
macro until(cond,blk)
    quote
        while true
            $blk
            if $cond
                break
            end
        end
    end
end

@until (macro with 1 method)

In [55]:
x = 10
@until x<=0 begin
    x -= 1
    println(x)
end

9
8
7
6
5
4
3
2
1
0


## Macro Hygiene

In [1]:
macro set_x(val)
    :(x = $val)
end

@set_x (macro with 1 method)

In [2]:
@set_x 10

10

In [3]:
x

LoadError: UndefVarError: x not defined

In [4]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

@set_x (macro with 1 method)

In [5]:
@set_x 10

10

In [6]:
x

10

## A macro approach to circular shifting

Recall the hand-specialized method for `s == 1`:

In [ ]:
function circularshift1!(X::AbstractVector)
    n = length(X)
    temp = X[n]
    temp2 = X[n-1]
    # ...
    copy!(X, 2, X, 1, n-1)
    X[1] = temp
    return X
end

In [ ]:
@eval function f(x, y, z)
    a = x
    b = 2x
    $(Symbol("foo"*"bar")) = 10
    ...
end

We can use a macro to generate extra temp variables for us.

In [8]:
macro load_n_temps(n::Int, arr, len)
    blk = Expr(:block)
    for i = 1:n
        var = esc(Symbol("_temp$i"))
        push!(blk.args, :($var = ($(esc(arr))[$len - $n + $i])))
    end
    blk
end

macro store_n_temps(n::Int, arr)
    blk = Expr(:block)
    for i = 1:n
        var = esc(Symbol("_temp$i"))
        push!(blk.args, :(($(esc(arr))[$i] = $var)))
    end
    blk
end

@store_n_temps (macro with 1 method)

In [9]:
function circularshift4!(X::AbstractVector)
    n = length(X)
    @load_n_temps 4 X n
    copy!(X, 5, X, 1, n-4)
    @store_n_temps 4 X
    return X
end

circularshift4! (generic function with 1 method)

In [10]:
circularshift4!(collect(1:10))

10-element Array{Int64,1}:
  7
  8
  9
 10
  1
  2
  3
  4
  5
  6

## Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro which is not exported is `@Base.Math.horner`, so in the current namespace, `horner` should be undefined:

In [ ]:
horner

In [12]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [13]:
x = 3
@horner(x, 2, 3, 4, 5)

182

To see what the macro does to this call, we again use `macroexpand`:

In [14]:
macroexpand(:(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

quote 
    #4#t = x
    2 + #4#t * (3 + #4#t * (4 + #4#t * (5 + #4#t * (6 + #4#t * (7 + #4#t * (8 + #4#t * (9 + #4#t * 10)))))))
end

In [15]:
macroexpand(:(@Base.Math.horner(x, 2, 3, 4, 5)))

quote 
    #5#t = x
    (Base.Math.muladd)(#5#t,(Base.Math.muladd)(#5#t,(Base.Math.muladd)(#5#t,5,4),3),2)
end

In [16]:
?muladd

search: muladd



```
muladd(x, y, z)
```

Combined multiply-add, computes `x*y+z` in an efficient manner. This may on some systems be equivalent to `x*y+z`, or to `fma(x,y,z)`. `muladd` is used to improve performance. See `fma`.


In [17]:
?fma

search: fma findmax findmax! @fastmath UniformScaling



```
fma(x, y, z)
```

Computes `x*y+z` without rounding the intermediate result `x*y`. On some systems this is significantly more expensive than `x*y+z`. `fma` is used to improve accuracy in certain algorithms. See `muladd`.


In [18]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [19]:
@code_llvm f(0.1)


define double @julia_f_71573(double) #0 {
top:
  %1 = call double @llvm.fmuladd.f64(double %0, double 4.500000e+00, double 3.400000e+00)
  %2 = call double @llvm.fmuladd.f64(double %0, double %1, double 2.300000e+00)
  %3 = call double @llvm.fmuladd.f64(double %0, double %2, double 1.200000e+00)
  ret double %3
}


In [20]:
@code_native f(0.1)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[18]
	pushq	%rbp
	movq	%rsp, %rbp
	movabsq	$4711182264, %rax       ## imm = 0x118CEEFB8
	movsd	(%rax), %xmm1           ## xmm1 = mem[0],zero
Source line: 1
	mulsd	%xmm0, %xmm1
	movabsq	$4711182272, %rax       ## imm = 0x118CEEFC0
	addsd	(%rax), %xmm1
	mulsd	%xmm0, %xmm1
	movabsq	$4711182280, %rax       ## imm = 0x118CEEFC8
	addsd	(%rax), %xmm1
	mulsd	%xmm1, %xmm0
	movabsq	$4711182288, %rax       ## imm = 0x118CEEFD0
	addsd	(%rax), %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [21]:
macroexpand(:(@evalpoly x 2 3 4 5))

:(let #14#tt = x # math.jl, line 111:
        if (Base.Math.isa)(#14#tt,Base.Math.Complex)
            #7#x = (Base.Math.real)(#14#tt)
            #8#y = (Base.Math.imag)(#14#tt)
            #9#r = #7#x + #7#x
            #10#s = (Base.Math.muladd)(#7#x,#7#x,#8#y * #8#y)
            #11#a2 = 5
            #12#a1 = (Base.Math.muladd)(#9#r,#11#a2,4)
            #13#a0 = (Base.Math.muladd)(#9#r,#12#a1,3 - #10#s * #11#a2)
            (Base.Math.muladd)(#13#a0,#14#tt,2 - #10#s * #12#a1)
        else 
            #15#t = #14#tt
            (Base.Math.muladd)(#15#t,(Base.Math.muladd)(#15#t,(Base.Math.muladd)(#15#t,5,4),3),2)
        end
    end)

In [22]:
g(x) = @evalpoly x 1.1 2.2 3.3

g (generic function with 1 method)

In [24]:
@code_llvm g(1.0 + 2.0im)


define void @julia_g_71625(%Complex.69* noalias sret, %Complex.69*) #0 {
top:
  %2 = getelementptr inbounds %Complex.69, %Complex.69* %1, i64 0, i32 0
  %3 = load double, double* %2, align 1
  %4 = getelementptr inbounds %Complex.69, %Complex.69* %1, i64 0, i32 1
  %5 = load double, double* %4, align 1
  %6 = fadd double %3, %3
  %7 = fmul double %5, %5
  %8 = call double @llvm.fmuladd.f64(double %3, double %3, double %7)
  %9 = call double @llvm.fmuladd.f64(double %6, double 3.300000e+00, double 2.200000e+00)
  %10 = load double, double* %2, align 8
  %11 = fmul double %8, 3.300000e+00
  %12 = fsub double 1.100000e+00, %11
  %13 = call double @llvm.fmuladd.f64(double %10, double %9, double %12)
  %14 = load double, double* %4, align 8
  %15 = fmul double %9, %14
  %16 = getelementptr inbounds %Complex.69, %Complex.69* %0, i64 0, i32 0
  store double %13, double* %16, align 8
  %17 = getelementptr inbounds %Complex.69, %Complex.69* %0, i64 0, i32 1
  store double %15, double* %17, ali

In [25]:
using BenchmarkTools

In [29]:
#function f()
#    x = 1.0; y = 2.0
    @benchmark x / y setup=(x=rand();y=rand())
#end

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  0.00 bytes
  allocs estimate:  0
  minimum time:     2.00 ns (0.00% GC)
  median time:      2.00 ns (0.00% GC)
  mean time:        2.00 ns (0.00% GC)
  maximum time:     20.00 ns (0.00% GC)

In [38]:
a = rand(2000,2000);
@time a .* 2.5;

  0.011235 seconds (6 allocations: 30.518 MB, 30.67% gc time)
